In [2]:
from splinter import Browser
from bs4 import BeautifulSoup as bs
import numpy as np
import pandas as pd
from selenium import webdriver
import requests as req

In [3]:
def init_browser():
    # @NOTE: Replace the path with your actual path to the chromedriver
    executable_path = {"executable_path":"/usr/local/bin/chromedriver"}
    return Browser("chrome", **executable_path, headless=False)

In [4]:
def close_browser(browser):
    browser.quit()

In [5]:
def scrape():
## initialize and populate mars dict to hold all scraping results 
    mars = {}
    mars["news_data"] = news()
    mars["featured_image"] = images()
    mars["weather_data"] = weather()
    mars["mars_facts"] = facts()

## Scrape Latest Mars News

In [6]:
## initialize browser and latest_news dict
browser = init_browser()

    
## define url and command browser to visit and parse HTML for posts tag   
nasa_url= "https://mars.nasa.gov"
news_url= "https://mars.nasa.gov/news/"
browser.visit(news_url)
    
html = browser.html
soup = bs(html, "html.parser")
news = soup.find_all('li', class_='slide')

# see result
print(f'raw: {news}')
print("---------------PROCESSING------------------")

## Create empty dict to hold data  
latest_news = {}

for new in news:
    ## Identify and return title of post
    title = soup.find('div', class_='content_title').text

    ## Identify and return date of post
    date = soup.find('div', class_='list_date').text

    ## Identify and return description to post
    description = soup.find('div', class_='article_teaser_body').text

    ## identify and return links to post
    link = soup.a['href']

## add results to latest_news dictionary
latest_news['news_title'] = title
latest_news['news_date'] = date
latest_news['news_about'] = description
latest_news["link"] = f'{nasa_url}{link}'


## close browser
close_browser(browser)

## print out dict to see result
print(latest_news)

raw: [<li class="slide"><div class="image_and_description_container"><a href="/news/8588/nine-finalists-chosen-in-nasas-mars-2020-rover-naming-contest/" target="_self"><div class="rollover_description"><div class="rollover_description_inner">Nine finalists have been chosen in the essay contest for K-12 students across U.S. to name NASA's next Mars rover. Now you can help by voting for your favorite. </div><div class="overlay_arrow"><img alt="More" src="/assets/overlay-arrow.png"/></div></div><div class="list_image"><img alt="This illustration depicts NASA's next Mars rover, which launches in 2020." src="/system/news_items/list_view_images/8588_name20190625-320x240.jpg"/></div><div class="bottom_gradient"><div><h3>Nine Finalists Chosen in NASA's Mars 2020 Rover Naming Contest</h3></div></div></a><div class="list_text"><div class="list_date">January 21, 2020</div><div class="content_title"><a href="/news/8588/nine-finalists-chosen-in-nasas-mars-2020-rover-naming-contest/" target="_self">

{'news_title': "Nine Finalists Chosen in NASA's Mars 2020 Rover Naming Contest", 'news_date': 'January 21, 2020', 'news_about': "Nine finalists have been chosen in the essay contest for K-12 students across U.S. to name NASA's next Mars rover. Now you can help by voting for your favorite. ", 'link': 'https://mars.nasa.govhttp://www.nasa.gov'}


## JPL Maars Spaace Images - Featured Image

In [7]:
##Initialize browser
browser = init_browser()

## create empty dictionary to hold results
marsimages = {}

## define image URL
image_url= "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(image_url)
    
##  get tags for Featured Image carousel
html = browser.html
soup = bs(html, "html.parser")
img_result = soup.find('div', class_='carousel_container')
    

## Identify and store featured image link
image_path = img_result.find('a', class_='button fancybox').get("data-fancybox-href")
featured_image_url= f'https://www.jpl.nasa.gov{image_path}'
## Dictionary to be inserted as a MongoDB document
marsimages["Featured Image URL"] = featured_image_url
            
## close browser
close_browser(browser)

## print out results
marsimages


{'Featured Image URL': 'https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA23170_ip.jpg'}

## Latest Mars Weather from Twitter

In [14]:
##Initialize browser
browser = init_browser()

## create empty dictionary to hold results
latest_weather = {}
    
## define twitter weather URL
weather_url= "https://twitter.com/marswxreport?lang=en"
browser.visit(weather_url)
html = browser.html
soup = bs(html, "html.parser")
w_tweet = soup.find('div', class_="js-tweet-text-container")
print(f'raw: {w_tweet}')
print("------------PROCESSING---------------")
wweather= w_tweet.find('p', class_="TweetTextSize TweetTextSize--normal js-tweet-text tweet-text").text

## apend to dictionary
latest_weather["Latest Mars Weather"] = wweather

# ##close browser
# close_browser(browser)

## print out dictionary
latest_weather

raw: None
------------PROCESSING---------------


AttributeError: 'NoneType' object has no attribute 'find'

## Mars Facts

In [19]:
mars_facts = {}
    
facts_url= "https://space-facts.com/mars/"
tables = pd.read_html(facts_url)
# print(f'raw: {tables}')
# print("-----------------------")

## create DF of mars facts table and convert to HTML
marsinfo_df = tables[0]
marsinfo_df.columns = ["Mars Planet Profile" , "Mars Data"]
marsinfo_df.set_index("Mars Planet Profile", inplace=True)
print(marsinfo_df)
marsfacts_df = marsinfo_df.to_html(classes = 'table table-striped')
# print(marsfacts_df)
marsfacts_data= marsinfo_df.to_dict()
print(marsfacts_data)


print("------------PROCESSING-------------")
##create DF of mars-earth comparison table and convert into HTML
comparison_df = tables[1]
comparison_df.columns = ["Mars-Earth Comparison" , "Mars Data","Earth Data"]
comparison_df.set_index("Mars-Earth Comparison", inplace=True)
print(comparison_df)
marsearth_df = comparison_df.to_html(classes = 'table table-striped')
marsearth_data= comparison_df.to_dict()
# print(marsearth_df)
print(marsearth_data)

mars_facts["mars data"] = marsfacts_data
mars_facts["mars earth comparison"] = marsearth_data
print("-----------PROCESSING-------------")
print(mars_facts)

                                          Mars Data
Mars Planet Profile                                
Equatorial Diameter:                       6,792 km
Polar Diameter:                            6,752 km
Mass:                 6.39 × 10^23 kg (0.11 Earths)
Moons:                          2 (Phobos & Deimos)
Orbit Distance:            227,943,824 km (1.38 AU)
Orbit Period:                  687 days (1.9 years)
Surface Temperature:                   -87 to -5 °C
First Record:                     2nd millennium BC
Recorded By:                   Egyptian astronomers
{'Mars Data': {'Equatorial Diameter:': '6,792 km', 'Polar Diameter:': '6,752 km', 'Mass:': '6.39 × 10^23 kg (0.11 Earths)', 'Moons:': '2 (Phobos & Deimos)', 'Orbit Distance:': '227,943,824 km (1.38 AU)', 'Orbit Period:': '687 days (1.9 years)', 'Surface Temperature:': '-87 to -5 °C', 'First Record:': '2nd millennium BC', 'Recorded By:': 'Egyptian astronomers'}}
--------------------------
                             Mars Dat

## Mars Hemisphere

In [ ]:
# ## create empty dict to hold data
# mars_hemispheres = {}
   
# ##initiate browser; define URL and visit in browser
# browser = init_browser()
# hemi_url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
# head_url = "https://astrogeology.usgs.gov"
# browser.visit(hemi_url)

# ##parse and scrape hemisphere info and image links
# html = browser.html
# soup = bs(html, "html.parser")
# hemi_results = soup.find('div', class_="collapsible results")
# hemispheres = hemi_results.find_all('div', class_="item")
# print(hemispheres)

In [ ]:
# ## Iterate through hemispheres list of results
# for hemi in hemispheres:
#     img_title = hemi.find('h3').text
#     print(img_title)
#     mars_hemispheres["Title"] = img_title
#     img_href = hemi.find('a', class_="itemLink product-item")['href']
#     hemi_img_url= (head_url)+(image_href)
#     print(hemi_img_url)
    
# #     browser.visit(hemi_img_url)
# ## Dictionary to be inserted as a MongoDB document
# marsimages["Featured Image URL"] = featured_image_url

In [28]:
## create empty dict to hold data
mars_hemispheres = []
   
##initiate browser; define URL and visit in browser
browser = init_browser()
hemi_url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
head_url = "https://astrogeology.usgs.gov"
browser.visit(hemi_url)


##parse and scrape hemisphere info and image links
html = browser.html
soup = bs(html, "html.parser")
hemispheres = soup.find_all('div', class_="item")
print(f'raw: {hemispheres}')
print("------------PROCESSING------------------")
    
for hemi in hemispheres:
#     browser.click_link_by_partial_href('/search/map/Mars/')
#     hemi_results = soup.find('div', class_="collapsible results")
    img_title = hemi.find('h3').text
#     print(img_title)
    hemi_img_url = head_url + hemi.find('a', class_="itemLink product-item")["href"]
#     hemi_img_url= f'{head_url}{img_href}'
## visit img url to get href for full-sized image
    browser.visit(hemi_img_url)
    url_to_full= browser.html
    soup = bs(url_to_full, "html.parser")
    full_img_url = head_url + soup.find("img", class_="wide-image")["src"]
    mars_hemispheres.append({"title": img_title, "img_url": full_img_url})

##close browser
close_browser(browser)

print(mars_hemispheres)

raw: [<div class="item"><a class="itemLink product-item" href="/search/map/Mars/Viking/cerberus_enhanced"><img alt="Cerberus Hemisphere Enhanced thumbnail" class="thumb" src="/cache/images/dfaf3849e74bf973b59eb50dab52b583_cerberus_enhanced.tif_thumb.png"/></a><div class="description"><a class="itemLink product-item" href="/search/map/Mars/Viking/cerberus_enhanced"><h3>Cerberus Hemisphere Enhanced</h3></a><span class="subtitle" style="float:left">image/tiff 21 MB</span><span class="pubDate" style="float:right"></span><br/><p>Mosaic of the Cerberus hemisphere of Mars projected into point perspective, a view similar to that which one would see from a spacecraft. This mosaic is composed of 104 Viking Orbiter images acquired…</p></div> <!-- end description --></div>, <div class="item"><a class="itemLink product-item" href="/search/map/Mars/Viking/schiaparelli_enhanced"><img alt="Schiaparelli Hemisphere Enhanced thumbnail" class="thumb" src="/cache/images/7677c0a006b83871b5a2f66985ab5857_sch